In [1]:
import numpy as np

from qiskit import BasicAer
from qiskit.transpiler import PassManager

from qiskit.aqua import Operator, QuantumInstance, aqua_globals
from qiskit.aqua.algorithms.adaptive import VQE
from qiskit.aqua.algorithms.classical import ExactEigensolver
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.aqua.components.variational_forms import RY

from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PyQuanteDriver, UnitsType, BasisType

aqua_globals.random_seed = 50

In [29]:
# using driver to get fermionic Hamiltonian
# PyQuante example
driver = PyQuanteDriver(atoms='H .0 .0 .0; H .0 .0 0.735', units=UnitsType.ANGSTROM,
                        charge=0, multiplicity=1, basis=BasisType.BSTO3G)
molecule = driver.run()
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals

In [30]:
h1

array([[-1.2563391,  0.       ,  0.       ,  0.       ],
       [ 0.       , -0.471896 ,  0.       ,  0.       ],
       [ 0.       ,  0.       , -1.2563391,  0.       ],
       [ 0.       ,  0.       ,  0.       , -0.471896 ]])

In [31]:
h2

array([[[[-0.33785508,  0.        ,  0.        ,  0.        ],
         [ 0.        , -0.0904656 ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]],

        [[ 0.        , -0.0904656 ,  0.        ,  0.        ],
         [-0.33229087,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [-0.33785508,  0.        ,  0.        ,  0.        ],
         [ 0.        , -0.0904656 ,  0.        ,  0.        ]],

        [[ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        , -0.0904656 ,  0.        ,  0.        ],
         [-0.33229087,  0.        ,  0.        , 

In [3]:
# convert from fermionic hamiltonian to qubit hamiltonian
ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_pa = ferOp.mapping(map_type='PARITY', threshold=0.00000001)
qubitOp_bk = ferOp.mapping(map_type='BRAVYI_KITAEV', threshold=0.00000001)

In [20]:
# print out qubit hamiltonian in Pauli terms and exact solution
qubit_ops = [(qubitOp_jw, 'jordan wigner'),
            (qubitOp_pa, 'parity'),
            (qubitOp_bk, 'bravyi-kitaev')]

for qubit_op, name in qubit_ops:
#    qubit_op.to_matrix()
    qubit_op.chop(10**-10)

    print("\n --- {} ---".format(name))
    print(qubit_op.print_details())

    # Using exact eigensolver to get the smallest eigenvalue
    exact_eigensolver = ExactEigensolver(qubit_op, k=1)
    ret = exact_eigensolver.run()
    print('The exact ground state energy using {} mapping is: {}'.format(name, ret['energy']))


 --- jordan wigner ---
IIII	(-0.8105479965981816+0j)
IIIZ	(0.1721839435195097+0j)
IIZI	(-0.2257535002794278+0j)
IZII	(0.17218394351950983+0j)
ZIII	(-0.2257535002794278+0j)
IIZZ	(0.12091263344524698+0j)
IZIZ	(0.1689275406336891+0j)
XXYY	(0.04523280037093533+0j)
YYYY	(0.04523280037093533+0j)
XXXX	(0.04523280037093533+0j)
YYXX	(0.04523280037093533+0j)
ZIIZ	(0.16614543381618233+0j)
IZZI	(0.16614543381618233+0j)
ZIZI	(0.17464343496146983+0j)
ZZII	(0.12091263344524698+0j)

The exact ground state energy using jordan wigner mapping is: -1.8572750815566383

 --- parity ---
IIII	(-0.8105479965981816+0j)
IIIZ	(0.1721839435195097+0j)
IIZZ	(-0.2257535002794278+0j)
IZZI	(0.17218394351950983+0j)
ZZII	(-0.2257535002794278+0j)
IIZI	(0.12091263344524698+0j)
IZZZ	(0.1689275406336891+0j)
IXZX	(-0.04523280037093533+0j)
ZXIX	(0.04523280037093533+0j)
IXIX	(0.04523280037093533+0j)
ZXZX	(-0.04523280037093533+0j)
ZZIZ	(0.16614543381618233+0j)
IZIZ	(0.16614543381618233+0j)
ZZZZ	(0.17464343496146983+0j)
ZIZI	(0.

In [21]:
for qubit_op, name in qubit_ops:
    # setup VQE 
    # setup optimizer, use L_BFGS_B optimizer for example
    lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

    # setup variational form generator (generate trial circuits for VQE)
    var_form = RY(qubit_op.num_qubits, 5, entanglement='full')

    # setup VQE with operator, variational form, and optimizer
    vqe_algorithm = VQE(qubit_op, var_form, lbfgs)

    backend = BasicAer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

    results = vqe_algorithm.run(quantum_instance)

    print("Ground state energy using {}: {}".format(name, results['eigvals'][0]))

Ground state energy using jordan wigner: -1.8572746568035747
Ground state energy using parity: -1.8572750814635517
Ground state energy using bravyi-kitaev: -1.8572750812150476


In [32]:
driver = PyQuanteDriver(atoms='H .0 .0 .0; Li .0 .0 0.735', units=UnitsType.ANGSTROM,
                        charge=0, multiplicity=1, basis=BasisType.BSTO3G)
molecule = driver.run()
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals

In [33]:
# convert from fermionic hamiltonian to qubit hamiltonian
ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_pa = ferOp.mapping(map_type='PARITY', threshold=0.00000001)
qubitOp_bk = ferOp.mapping(map_type='BRAVYI_KITAEV', threshold=0.00000001)

In [34]:
h1

array([[-5.1034254 , -0.17681234,  0.1521196 ,  0.        ,  0.        ,
         0.16305696,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-0.17681234, -1.80510711, -0.06161436,  0.        ,  0.        ,
         0.49913303,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.1521196 , -0.06161436, -1.20468521,  0.        ,  0.        ,
         0.03196874,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -1.22753134,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        , -1.22753134,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.16305696,  0.49913303,  0.03196874,  0.        ,  0.        ,
        -1.11954486,  

In [35]:
h2

array([[[[-8.09051202e-01, -9.59009558e-02,  5.11520053e-02, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [-9.59009558e-02, -2.51901364e-02,  5.56386148e-03, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 5.11520053e-02,  5.56386148e-03, -7.85554832e-03, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[-9.59009558e-02, -2.51901364e-02,  5.56386148e-03, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [-2.62055334e-01,  7.49494019e-03,  1.43763843e-02, ...,
           0.00000000e

In [36]:
# print out qubit hamiltonian in Pauli terms and exact solution
qubit_ops = [(qubitOp_jw, 'jordan wigner'),
            (qubitOp_pa, 'parity'),
            (qubitOp_bk, 'bravyi-kitaev')]

for qubit_op, name in qubit_ops:
#    qubit_op.to_matrix()
    qubit_op.chop(10**-10)

    print("\n --- {} ---".format(name))
    print(qubit_op.print_details())

    # Using exact eigensolver to get the smallest eigenvalue
    exact_eigensolver = ExactEigensolver(qubit_op, k=1)
    ret = exact_eigensolver.run()
    print('The exact ground state energy using {} mapping is: {}'.format(name, ret['energy']))


 --- jordan wigner ---
IIIIIIIIIIII	(-5.859185964100336+0j)
IIIIIIIIIIIZ	(1.084627295592738+0j)
IIIIIIIIIIYY	(-0.02907623324081622+0j)
IIIIIIIIIIXX	(-0.02907623324081622+0j)
IIIIIIIIIYZY	(0.02055698786598697+0j)
IIIIIIIIIXZX	(0.02055698786598697+0j)
IIIIIIYZZZZY	(0.03321885317468601+0j)
IIIIIIXZZZZX	(0.03321885317468601+0j)
IIIIIIIIIIZI	(-0.09133228312536135+0j)
IIIIIIIIIYYI	(0.007846471440114935+0j)
IIIIIIIIIXXI	(0.007846471440114935+0j)
IIIIIIYZZZYI	(0.022147427517145966+0j)
IIIIIIXZZZXI	(0.022147427517145966+0j)
IIIIIIIIIZII	(-0.17924373104636407+0j)
IIIIIIYZZYII	(0.012323562832007154+0j)
IIIIIIXZZXII	(0.012323562832007154+0j)
IIIIIIIIZIII	(-0.21106778238544618+0j)
IIIIIIIZIIII	(-0.2110677823854461+0j)
IIIIIIZIIIII	(-0.3766423764826785+0j)
IIIIIZIIIIII	(1.084627295592739+0j)
IIIIYYIIIIII	(-0.029076233240816183+0j)
IIIIXXIIIIII	(-0.029076233240816183+0j)
IIIYZYIIIIII	(0.02055698786598697+0j)
IIIXZXIIIIII	(0.02055698786598697+0j)
YZZZZYIIIIII	(0.03321885317468597+0j)
XZZZZXIIIIII	(0.

The exact ground state energy using jordan wigner mapping is: -9.715591572061571

 --- parity ---
IIIIIIIIIIII	(-5.859185964100336+0j)
IIIIIIIIIIIZ	(1.084627295592738+0j)
IIIIIIIIIIZX	(0.02907623324081622+0j)
IIIIIIIIIIIX	(-0.02907623324081622+0j)
IIIIIIIIIZXX	(-0.02055698786598697+0j)
IIIIIIIIIIYY	(-0.02055698786598697+0j)
IIIIIIZXXXXX	(-0.03321885317468601+0j)
IIIIIIIYXXXY	(-0.03321885317468601+0j)
IIIIIIIIIIZZ	(-0.09133228312536135+0j)
IIIIIIIIIZXZ	(-0.007846471440114935+0j)
IIIIIIIIIIXI	(0.007846471440114935+0j)
IIIIIIZXXXXZ	(-0.022147427517145966+0j)
IIIIIIIYXXYI	(-0.022147427517145966+0j)
IIIIIIIIIZZI	(-0.17924373104636407+0j)
IIIIIIZXXXZI	(-0.012323562832007154+0j)
IIIIIIIYXYII	(-0.012323562832007154+0j)
IIIIIIIIZZII	(-0.21106778238544618+0j)
IIIIIIIZZIII	(-0.2110677823854461+0j)
IIIIIIZZIIII	(-0.3766423764826785+0j)
IIIIIZZIIIII	(1.084627295592739+0j)
IIIIZXZIIIII	(0.029076233240816183+0j)
IIIIIXIIIIII	(-0.029076233240816183+0j)
IIIZXXZIIIII	(-0.02055698786598697+0j)
IIIIYYIIII

The exact ground state energy using parity mapping is: -9.715591572061546

 --- bravyi-kitaev ---
IIIIIIIIIIII	(-5.859185964100336+0j)
IIIIIIIIIIIZ	(1.084627295592738+0j)
IIIIIIIIIIZX	(0.02907623324081622+0j)
IIIIIIIIIIIX	(-0.02907623324081622+0j)
IIIIIIIIIYYX	(0.02055698786598697+0j)
IIIIIIIIIXYY	(-0.02055698786598697+0j)
IIIIIIYIYIXX	(0.03321885317468601+0j)
IIIIIIXZYIXY	(-0.03321885317468601+0j)
IIIIIIIIIIZZ	(-0.09133228312536135+0j)
IIIIIIIIIYYZ	(0.007846471440114935+0j)
IIIIIIIIIXXI	(0.007846471440114935+0j)
IIIIIIYIYIXZ	(0.022147427517145966+0j)
IIIIIIXZYIYI	(-0.022147427517145966+0j)
IIIIIIIIIZII	(-0.17924373104636407+0j)
IIIIIIYIYXZI	(0.012323562832007154+0j)
IIIIIIXZYYZI	(-0.012323562832007154+0j)
IIIIIIIIZZZI	(-0.21106778238544618+0j)
IIIIIIIZIIII	(-0.2110677823854461+0j)
IIIIIIZZIIII	(-0.3766423764826785+0j)
IIIIIZIIIIII	(1.084627295592739+0j)
IIIIZXZIZIII	(0.029076233240816183+0j)
IIIIIXIIIIII	(-0.029076233240816183+0j)
XIXYYXZIZIII	(0.02055698786598697+0j)
XIXXYYZIZIII	(-0

The exact ground state energy using bravyi-kitaev mapping is: -9.715591572061577


In [37]:
for qubit_op, name in qubit_ops:
    # setup VQE 
    # setup optimizer, use L_BFGS_B optimizer for example
    lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

    # setup variational form generator (generate trial circuits for VQE)
    var_form = RY(qubit_op.num_qubits, 5, entanglement='full')

    # setup VQE with operator, variational form, and optimizer
    vqe_algorithm = VQE(qubit_op, var_form, lbfgs)

    backend = BasicAer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

    results = vqe_algorithm.run(quantum_instance)

    print("Ground state energy using {}: {}".format(name, results['eigvals'][0]))

Ground state energy using jordan wigner: -9.372483457476335
Ground state energy using parity: -9.2081016031359
Ground state energy using bravyi-kitaev: -9.278611753456829
